# Using the Bot Service API programmatically

In the previous notebook, we developed a Backend Web API utilizing the Bot Framework, which is hosted on the Azure Web App service.

Additionally, we crafted a demonstration Front End using Streamlit, incorporating a JavaScript snippet to display an engaging chat window. This chat interface can be seamlessly embedded and customized as needed.

In this notebook, we will programmatically interact with this API using pure Python code through the Direct Line Channel.

In [1]:
import os
import aiohttp
import asyncio
import time 
import requests
import datetime
import pytz

import nest_asyncio
nest_asyncio.apply()  # this is only needed for jupyter notebooks

from dotenv import load_dotenv
load_dotenv("credentials.env")

# Declare Bot Service variables

base_url = os.environ["BOT_DIRECT_CHANNEL_ENDPOINT"]
bot_id = os.environ["BOT_ID"]
direct_line_secret = os.environ["BOT_SERVICE_DIRECT_LINE_SECRET"]

timeout = 25

Regarding the Bot Service Direct Line channel, there are two primary endpoints of interest:

- `{base_url}/conversations`: Initiates the conversation and provides the conversation ID.
- `{base_url}/conversations/{conversation_id}/activities`: Asynchronously returns every activity occurring within the bot.

The fundamental workflow involves:
1) Initiating a new conversation by utilizing the `/conversations` endpoint.
2) Sending messages through the `/conversations/{conversation_id}/activities` endpoint.
3) Periodically polling the `/conversations/{conversation_id}/activities` endpoint to retrieve responses, errors, and other relevant activities.


In [2]:
%%time

# Simple workflow
start_conversation_url = f"{base_url}/conversations"

# 1- Start a conversation
headers = {"Authorization": f"Bearer {direct_line_secret}"}
response = requests.post(start_conversation_url, headers=headers)
conversation_id = response.json()["conversationId"]
print('Converstion id:', conversation_id)

get_activities_url = f"{base_url}/conversations/{conversation_id}/activities"


# 2 - Send a message to the bot
send_message_url = f"{base_url}/conversations/{conversation_id}/activities"
message = {
    "type": "message",
    "from": {"id": "user"},
    "text": "what is CLP?"
}

response = requests.post(send_message_url, headers=headers, json=message)

# 3 - Wait a bit and Get Activities
time.sleep(5)
response = requests.get(get_activities_url, headers=headers)
activities = response.json()["activities"]

Converstion id: 9wah6oavbyqHdVAOR5hBEx-au
CPU times: user 130 ms, sys: 4.95 ms, total: 135 ms
Wall time: 11.6 s


In [3]:
activities

[{'type': 'message',
  'id': '9wah6oavbyqHdVAOR5hBEx-au|0000000',
  'timestamp': '2024-07-10T11:11:04.50885Z',
  'channelId': 'directline',
  'from': {'id': 'BotId-zf4fwhz3gdn64', 'name': 'BotId-zf4fwhz3gdn64'},
  'conversation': {'id': '9wah6oavbyqHdVAOR5hBEx-au'},
  'text': '\nHello and welcome! 👋\n\nMy name is Jarvis, a smart virtual assistant designed to assist you.\nHere\'s how you can interact with me:\n\nI have various plugins and tools at my disposal to answer your questions effectively. Here are the available options:\n\n1. 🌐 **bing**: This tool allows me to access the internet and provide current information from the web.\n\n2. 💡 **chatgpt**: With this tool, I can draw upon my own knowledge based on the data I was trained on. Please note that my training data goes up until 2021.\n\n3. 🔍 **docsearch**: This tool allows me to search a specialized search engine index. It includes 10,000 ArXiv computer science documents from 2020-2021 and 90,000 Covid research articles from the s

## Create helper functions to talk to the API asyncronously

These functions below define a simple system for asynchronously sending a message to a bot, waiting for a response, and then continuously checking for and printing new messages from the bot for a specified period. It uses aiohttp for asynchronous HTTP requests, allowing it to non-blockingly wait for responses from the bot and enforce a timeout if no new messages are received within the expected timeframe.

Modify these at your will

In [4]:
# Function to send a message to the bot service API.
async def send_message(base_url, conversation_id, headers, question):
    # Construct the URL for sending a message to the bot.
    send_message_url = f"{base_url}/conversations/{conversation_id}/activities"
    
    # Prepare the timestamp, timezone, and locale for the message.
    local_timestamp = datetime.datetime.now(pytz.timezone('America/New_York'))
    local_timezone = str(local_timestamp.tzinfo)
    locale = "en-US"

    # Define the message payload, including the question and additional data.
    message = {
        "type": "message",
        "from": {"id": "user"},
        "text": question,
        "channelData": {
            "local_timestamp": local_timestamp.strftime("%I:%M:%S %p, %A, %B %d of %Y"),
            "local_timezone": local_timezone,
            "locale": locale
        },
        # Example structure for sending an attachment, commented out here.
        # "attachments": [
        #     {
        #         "contentType": "image/jpeg",
        #         "contentUrl": "https://example.com/image.jpg",
        #         "name": "image.jpg"
        #     }
        # ]
    }
    
    # Use an asynchronous HTTP session to send the message.
    async with aiohttp.ClientSession() as session:
        async with session.post(send_message_url, headers=headers, json=message) as response:
            print("Message sent status code:", response.status)
            response_text = await response.text()
            print("Response text:", response_text)

            
# Function to filter and print the last bot responses to the most recent user message.
async def print_last_bot_responses(activities, bot_id):
    last_user_msg_index = None
    # Iterate through activities in reverse to find the last user message.
    for index, msg in enumerate(reversed(activities)):
        if msg['from']['id'] != bot_id:
            last_user_msg_index = len(activities) - 1 - index
            break

    messages_to_print = []
    # If a user message was found, collect all subsequent bot messages.
    if last_user_msg_index is not None:
        for msg in activities[last_user_msg_index + 1:]:
            if msg['from']['id'] == bot_id:
                messages_to_print.append(msg['text'])
    
    return messages_to_print


# Main function to send a question to the bot and print responses.
async def send_question_and_check_activities(base_url, bot_id, conversation_id, headers, question, timeout=30):
    # Send the initial question to the bot.
    await send_message(base_url, conversation_id, headers, question)
    
    async with aiohttp.ClientSession() as session:
        last_printed_activity_id = None
        # Record the time when the last message was received to enforce the timeout.
        last_message_time = time.time()

        while True:
            current_time = time.time()
            # Check if the specified timeout has elapsed without new messages.
            if current_time - last_message_time > timeout:
                print(f"{timeout} seconds have elapsed without new messages. Exiting...")
                break

            # Construct the URL to get conversation activities.
            get_activities_url = f"{base_url}/conversations/{conversation_id}/activities"
            
            # Use an asynchronous HTTP session to fetch activities.
            async with session.get(get_activities_url, headers=headers) as response:
                activities = await response.json()
                activities = activities["activities"]
                new_messages = await print_last_bot_responses(activities, bot_id)
                
                # Check for new messages from the bot since the last printed message.
                if new_messages:
                    last_activity_id = activities[-1]['id']
                    if last_activity_id != last_printed_activity_id:
                        for message in new_messages:
                            print(message)
                        # Update tracking variables with the latest message details.
                        last_printed_activity_id = last_activity_id
                        last_message_time = current_time

            # Wait for a short period before checking for new messages again.
            await asyncio.sleep(1)


## Talk to the bot API 

In [5]:
# Start a conversation
start_conversation_url = f"{base_url}/conversations"
headers = {"Authorization": f"Bearer {direct_line_secret}"}

response = requests.post(start_conversation_url, headers=headers)
conversation_id = response.json()["conversationId"]
print('Converstion id:', conversation_id)


Converstion id: II0A5i5aSjXHN8VsJuildi-au


### Ask the first question

In [6]:
QUESTION = "sqlsearch, what is the country with the most deaths in 2020?"

In [7]:
await send_question_and_check_activities(base_url, bot_id, conversation_id, headers, QUESTION, timeout=timeout)

Message sent status code: 200
Response text: {
  "id": "II0A5i5aSjXHN8VsJuildi-au|0000001"
}
Tool: sqlsearch
☑
Invoking: `sql_db_list_tables` with `{}`


 ...
☑
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`


 ...
☑
Invoking: `sql_db_query` with `{'query': "SELECT state, SUM(death) AS total_deaths FROM covidtracking WHERE date LIKE '2020%' GROUP BY state ORDER BY total_deaths DESC"}`


 ...
The state with the most deaths in 2020 was New York (NY), with a total of 6,530,995 deaths.

This information was obtained by querying the `covidtracking` table for the `state` and the sum of `death` where the date starts with '2020'. The results were then grouped by state and ordered by the total number of deaths in descending order. The SQL query used for this purpose is as follows:

```sql
SELECT state, SUM(death) AS total_deaths
FROM covidtracking
WHERE date LIKE '2020%'
GROUP BY state
ORDER BY total_deaths DESC
```

This query returned the total number of deaths for each sta

### Now a follow up question

In [8]:
FOLLOWUP_QUESTION = "interesting, and about the state with the least?"

In [9]:
await send_question_and_check_activities(base_url, bot_id, conversation_id, headers, FOLLOWUP_QUESTION, timeout=timeout)

Message sent status code: 200
Response text: {
  "id": "II0A5i5aSjXHN8VsJuildi-au|0000007"
}
Tool: sqlsearch
☑
Invoking: `sql_db_list_tables` with `{}`


 ...
☑
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`


 ...
☑
Invoking: `sql_db_query` with `{'query': "SELECT state, SUM(death) AS total_deaths FROM covidtracking WHERE date LIKE '2020%' GROUP BY state ORDER BY total_deaths ASC"}`


 ...
The state with the lowest total deaths from COVID-19 in 2020 was American Samoa (AS) with 0 deaths.

This information was obtained by querying the `covidtracking` table for the `state` and the sum of `death` where the date starts with '2020'. The results were then grouped by state and ordered by the total number of deaths in ascending order. The SQL query used for this purpose is as follows:

```sql
SELECT state, SUM(death) AS total_deaths
FROM covidtracking
WHERE date LIKE '2020%'
GROUP BY state
ORDER BY total_deaths ASC
LIMIT 1
```

This query returned the state with the lowest 

# Summary

In this notebook, we've explored how to programmatically communicate with the Bot Service API.

An important aspect to note regarding the responses from the API is that it consistently returns the entire conversation history, not just the latest message. This behavior is evident in the following code snippet from the helper functions mentioned earlier:

```python
async with session.get(get_activities_url, headers=headers) as response:
                activities = await response.json()
                activities = activities["activities"]
                new_messages = await print_last_bot_responses(activities, bot_id)
```
Go ahead and print out the variable `activities`; you'll observe this for yourself.

This characteristic opens up the possibility of developing a simpler memory system. By including previous messages along with the current question, it's feasible to manage conversational context using the frontend without the need for persistent storage solutions like Cosmos DB in the backend.

# NEXT

In the next notebook, we will venture into creating a different type of Backend API, this time utilizing FastAPI and LangServe. This approach will also enable us to incorporate streaming capabilities.